In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [5]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [6]:
X, Y = load_coffee_data()
print(X.shape, Y.shape)

(200, 2) (200, 1)


In [7]:
print(f"Temperature Max, Min pre Normalization: {np.max(X[:, 0]):0.2f}, {np.min(X[:, 0]):0.2f}")
print(f"Duration    Max, Min pre Normalization: {np.max(X[:, 1]):0.2f}, {np.min(X[:, 1]):0.2f}")

Temperature Max, Min pre Normalization: 284.99, 151.32
Duration    Max, Min pre Normalization: 15.45, 11.51


In [8]:
# Normalization of the Data
norm_1 = tf.keras.layers.Normalization(axis=-1)
# adapt() is used to learn mean and variance from the data
norm_1.adapt(X)
Xn = norm_1(X)

In [9]:
print(f"Temperature Max, Min Post Normalization: {np.max(Xn[:, 0]):0.2f}, {np.min(Xn[:, 0]):0.2f}")
print(f"Duration    Max, Min post Normalization: {np.max(Xn[:, 1]):0.2f}, {np.min(Xn[:, 1]):0.2f}")

Temperature Max, Min Post Normalization: 1.66, -1.69
Duration    Max, Min post Normalization: 1.79, -1.70


In [10]:
# Increase the Current Dataset
Xt = np.tile(Xn, (1000, 1))
Yt = np.tile(Y, (1000, 1))
print(Xt.shape, Yt.shape)

(200000, 2) (200000, 1)


In [13]:
# Build a Model
tf.random.set_seed(1234) # To Get Consistent Results
model = Sequential(
    [
        # Specify the Shape of the Input
        tf.keras.Input(shape=(2,)),
        # Layer 1
        Dense(3, activation='sigmoid', name='layer1'),
        # Layer 2
        Dense(1, activation='sigmoid', name='layer2')
    ]
)

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Weights before Training
w1, b1 = model.get_layer('layer1').get_weights()
w2, b2 = model.get_layer('layer2').get_weights()
print(f"W1{w1.shape}:\n", w1, f"\nb1{b1.shape}:", b1)
print(f"W2{w2.shape}:\n", w2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.83701587 -0.81385285 -0.00218916]
 [-0.01571202 -0.40241843 -0.48458833]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.86867166]
 [ 0.90148103]
 [ 0.44986582]] 
b2(1,): [0.]


In [19]:
# Compile the Model with Loss and Optimizer
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

In [20]:
# Fit the Model with the Data
model.fit(
    Xt, Yt,
    epochs=10
)

Epoch 1/10
6250/6250 [==============================] - 49s 7ms/step - loss: 0.1807
Epoch 2/10
6250/6250 [==============================] - 46s 7ms/step - loss: 0.0962
Epoch 3/10
6250/6250 [==============================] - 43s 7ms/step - loss: 0.0247
Epoch 4/10
6250/6250 [==============================] - 38s 6ms/step - loss: 0.0132
Epoch 5/10
6250/6250 [==============================] - 50s 8ms/step - loss: 0.0087
Epoch 6/10
6250/6250 [==============================] - 44s 7ms/step - loss: 0.0060
Epoch 7/10
6250/6250 [==============================] - 44s 7ms/step - loss: 0.0041
Epoch 8/10
6250/6250 [==============================] - 33s 5ms/step - loss: 0.0028
Epoch 9/10
6250/6250 [==============================] - 49s 8ms/step - loss: 0.0020
Epoch 10/10
6250/6250 [==============================] - 46s 7ms/step - loss: 0.0014


In [21]:
# Weights After Training
w1, b1 = model.get_layer('layer1').get_weights()
w2, b2 = model.get_layer('layer2').get_weights()
print(f"W1{w1.shape}:\n", w1, f"\nb1{b1.shape}:", b1)
print(f"W2{w2.shape}:\n", w2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 14.6055       0.20452468 -11.289298  ]
 [ 12.111599    10.4002285   -0.31029156]] 
b1(3,): [  2.0063815  12.537889  -12.139562 ]
W2(3, 1):
 [[-47.93582 ]
 [ 46.120235]
 [-56.190105]] 
b2(1,): [-14.355996]


In [23]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = norm_1(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 1s 1s/step
predictions = 
 [[9.939981e-01]
 [9.273377e-08]]


In [24]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[1]
 [0]]
